In [1]:
import os
import sys
import h5py
import numpy as np
import torch as pt
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import roc_auc_score

import mdtraj_utils as mdu
from CLoNe.clone import CLoNe
from src.structure import clean_structure, tag_hetatm_chains, split_by_chain, filter_non_atomic_subunits, remove_duplicate_tagged_subunits, concatenate_chains, atom_select, data_to_structure, encode_bfactor
from src.data_encoding import config_encoding, encode_structure, encode_features, extract_topology, extract_all_contacts, std_elements, std_resnames, std_names
from src.dataset import collate_batch_features
from src.structure_io import save_pdb

In [2]:
def traj_to_struct(traj):
    df = traj.topology.to_dataframe()[0]
    return {
        "xyz": np.transpose(traj.xyz, (1,0,2))*1e1,
        "name": df["name"].values,
        "element": df["element"].values,
        "resname": df["resName"].values,
        "resid": df["resSeq"].values,
        "het_flag": np.array(['A']*traj.xyz.shape[1]),
        "chain_name": df["chainID"].values,
        "icode": np.array([""]*df.shape[0]),
    }


def process_structure(structure):
    # process structure
    structure = clean_structure(structure)

    # update molecules chains
    structure = tag_hetatm_chains(structure)

    # split structure
    subunits = split_by_chain(structure)

    # remove non atomic structures
    subunits = filter_non_atomic_subunits(subunits)

    # remove duplicated molecules and ions
    subunits = remove_duplicate_tagged_subunits(subunits)
    
    return subunits


def superpose_transform(xyz_ref, xyz):
    # centering
    t = np.expand_dims(np.mean(xyz,axis=1),1)
    t_ref = np.expand_dims(np.mean(xyz_ref,axis=1),1)

    # SVD decomposition
    U, S, Vt = np.linalg.svd(np.matmul(np.swapaxes(xyz_ref-t_ref,1,2), xyz-t))

    # reflection matrix
    Z = np.zeros(U.shape) + np.expand_dims(np.eye(U.shape[1], U.shape[2]),0)
    Z[:,-1,-1] = np.linalg.det(U) * np.linalg.det(Vt)

    R = np.matmul(np.swapaxes(Vt,1,2), np.matmul(Z, np.swapaxes(U,1,2)))

    return t_ref, t, R


def superpose(xyz_ref, xyz):
    # centering
    t = np.expand_dims(np.mean(xyz,axis=1),1)
    t_ref = np.expand_dims(np.mean(xyz_ref,axis=1),1)

    # SVD decomposition
    U, S, Vt = np.linalg.svd(np.matmul(np.swapaxes(xyz_ref-t_ref,1,2), xyz-t))

    # reflection matrix
    Z = np.zeros(U.shape) + np.expand_dims(np.eye(U.shape[1], U.shape[2]),0)
    Z[:,-1,-1] = np.linalg.det(U) * np.linalg.det(Vt)

    R = np.matmul(np.swapaxes(Vt,1,2), np.matmul(Z, np.swapaxes(U,1,2)))

    return xyz_ref-t_ref, np.matmul(xyz-t, R)

In [3]:
# model parameters
# R3
#save_path = "save/i_v3_0_2021-05-27_14-27"  # 89
#save_path = "save/i_v3_1_2021-05-28_12-40"  # 90
# R4
#save_path = "save/i_v4_0_2021-09-07_11-20"  # 89
save_path = "save/i_v4_1_2021-09-07_11-21"  # 91

# select saved model
model_filepath = os.path.join(save_path, 'model_ckpt.pt')
#model_filepath = os.path.join(save_path, 'model.pt')

In [4]:
# add module to path
if save_path not in sys.path:
    sys.path.insert(0, save_path)
    
# load functions
from config import config_model, config_data
from data_handler import Dataset
from model import Model

In [5]:
# define device
device = pt.device("cuda")

# create model
model = Model(config_model)

# reload model
model.load_state_dict(pt.load(model_filepath, map_location=pt.device("cpu")))

# set model to inference
model = model.eval().to(device)

In [6]:
# parameters
pdbids = ["1JTG","1CLV","1Z0K","1AK4","1R6Q","1D6R","2I25","3F1P","1R0R","1E96","1GPW","1RKE","1FLE","2O3B","3SGQ","1ZHH","1CGI","2UUY","2HQS","2OOB"]
mdids = ["uR", "uL"]

# setup data connector
dc = mdu.data.DataConnector("database")

In [7]:
# parameters
n_skip = 100

for pdbid in pdbids:
    # load reference
    dc.load_reference(pdbid, "C")

    # convert and process structure
    struct_ref = traj_to_struct(dc[pdbid]["C"]["traj_ref"])
    struct_ref['xyz'] = struct_ref['xyz'][:,0]
    subunits_ref = process_structure(struct_ref)

    # find interfaces
    contacts = extract_all_contacts(subunits_ref, 5.0, device=device)
    
    # for each md
    for mdid in mdids:
        # debug print
        print(pdbid, mdid)

        # load trajectory
        dc.load_trajectory(pdbid, mdid)

        # convert to structure
        structure = traj_to_struct(dc[pdbid][mdid]['traj'])

        # process structure
        subunits = process_structure(structure)

        # concatenate subunits
        structure = concatenate_chains(subunits)

        # encode structure and features
        X_traj, M = encode_structure(structure)
        q_all = pt.cat(encode_features(structure), dim=1)
        q = encode_features(structure)[0]

        # extract topology
        ids_topk, D_topk, R_topk, D, R = extract_topology(X_traj[:,0], 64)

        # pack data and setup sink (IMPORTANT)
        _, ids_topk, q, M = collate_batch_features([[X_traj[:,0], ids_topk, q, M]])
        
        # auto-detect chains
        ids_sim = mdu.utils.align(dc[pdbid]["C"]["traj_ref"], dc[pdbid][mdid]["traj"], selection="all")
        cids_ref = dc[pdbid]["C"]["traj_ref"].topology.to_dataframe()[0].iloc[ids_sim[:,0]]['chainID'].unique().astype('str')
        cids = np.array(list(contacts))
        
        # define labels
        ids = contacts[cids_ref[0]][cids[~np.isin(cids, cids_ref)][0]]['ids']
        y = np.zeros(M.shape[0])
        y[ids[:,0]] = 1.0
        y = (np.matmul(y, M.detach().cpu().numpy()) > 0.5).astype(float)

        # run model
        P, t = [], []
        with pt.no_grad():
            for i in tqdm(range(0, X_traj.shape[1], n_skip)):
                # extract frame coordinates
                X = X_traj[:,i]

                # make prediction
                z = model(X.to(device), ids_topk.to(device), q.to(device), M.float().to(device))
                #p = pt.sigmoid(z).flatten()
                p = pt.sigmoid(z)[:,0].flatten()

                # store results
                P.append(p.detach().cpu().numpy())
                t.append(dc[pdbid][mdid]['traj'].time[i])
            
        # get atom coordinates for C_alpha for predicted frames
        X_traj_slice = X_traj[:, pt.arange(0, X_traj.shape[1], n_skip)]
        Xp = (pt.matmul(X_traj_slice.transpose(0,2), M) / pt.sum(M, axis=0).reshape(1,1,-1)).transpose(0,2).transpose(0,1).numpy()
        _, Xp = superpose(np.expand_dims(Xp[0],0), Xp)

        # pack results
        P = np.array(P)
        t = np.array(t)
        
        # save results
        np.savez("outputs/{}_{}.npz".format(pdbid, mdid), P=P, t=t, Xp=Xp, y=y)
        
        # unload data
        dc.unload_md(pdbid, mdid)
        
    # unload data
    dc.unload_pdb(pdbid)

1JTG bR


100%|██████████████████████████████████████████████████████████████████████████████| 534/534 [00:40<00:00, 13.10it/s]


1JTG bL


100%|██████████████████████████████████████████████████████████████████████████████| 546/546 [01:04<00:00,  8.51it/s]


1CLV bR


...

100%|██████████████████████████████████████████████████████████████████████████████| 544/544 [00:31<00:00, 17.34it/s]


2OOB bR


100%|██████████████████████████████████████████████████████████████████████████████| 552/552 [00:20<00:00, 26.77it/s]


2OOB bL


100%|██████████████████████████████████████████████████████████████████████████████| 502/502 [00:22<00:00, 22.26it/s]
